## LLM Zoomcamp 

### HW1

### Q1 - Running Elastic Search

In [1]:
from elasticsearch import Elasticsearch

# Connect to Elasticsearch (adjust connection settings as needed)
es = Elasticsearch("http://localhost:9200")

# Get cluster health
health = es.cluster.health()
print("Cluster health:", health)

# Get cluster information
info = es.info()
print("Cluster info:", info)

# Get nodes information
nodes = es.nodes.info()
print("Nodes info:", nodes)

Cluster health: {'cluster_name': 'docker-cluster', 'status': 'green', 'timed_out': False, 'number_of_nodes': 1, 'number_of_data_nodes': 1, 'active_primary_shards': 1, 'active_shards': 1, 'relocating_shards': 0, 'initializing_shards': 0, 'unassigned_shards': 0, 'unassigned_primary_shards': 0, 'delayed_unassigned_shards': 0, 'number_of_pending_tasks': 0, 'number_of_in_flight_fetch': 0, 'task_max_waiting_in_queue_millis': 0, 'active_shards_percent_as_number': 100.0}
Cluster info: {'name': '45c97701ffbf', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'VkLCtS6fSHmzOmz3aTAzeA', 'version': {'number': '8.17.6', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'dbcbbbd0bc4924cfeb28929dc05d82d662c527b7', 'build_date': '2025-04-30T14:07:12.231372970Z', 'build_snapshot': False, 'lucene_version': '9.12.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}
Nodes info: {'_nodes': {'total': 1, 'succes

"build_hash" : "dbcbbbd0bc4924cfeb28929dc05d82d662c527b7",

In [2]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [3]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es.indices.create(index=index_name, body=index_settings)

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [course-questions/qy2GKEKZQ6CW3FBL8Z8oog] already exists')

In [4]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

### Q2 - Indexing the data

Which function do you use for adding your data to elastic?

- index

In [6]:
from tqdm.auto import tqdm
for doc in tqdm(documents):
    es.index(index=index_name, document=doc)

d:\DataScience\LLM_Zoomcamp\llm-zoomcamp\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 948/948 [00:47<00:00, 20.04it/s]


### Q3. Searching

Now let's search in our index. 

We will execute a query "How do execute a command on a Kubernetes pod?". 

Use only `question` and `text` fields and give `question` a boost of 4, and use `"type": "best_fields"`.

What's the score for the top ranking result?

* 84.50
* 64.50
* 44.50
* 24.50

Look at the `_score` field.

In [7]:
query = 'How do execute a command on a Kubernetes pod?'

In [8]:
def elastic_search(query):
    search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
        }
    }
}

    response = es.search(index=index_name, body=search_query)
    return response    
    # result_docs = []
    
    # for hit in response['hits']['hits']:
    #     result_docs.append(hit['_source'])
    
    # return result_docs

In [ ]:
response = elastic_search(query)
for hit in response['hits']['hits']:
    print(f"Score: {hit['_score']}, Course: {hit['_source']['course']}, Question: {hit['_source']['question']}")
    print(f"Text: {hit['_source']['text']}\n")

Score: 44.50556, Course: machine-learning-zoomcamp, Question: How do I debug a docker container?
Text: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Score: 35.433445, Course: machine-learning-zoomcamp, Question: Kubernetes-dashboard
Text: Deploy and Access the Kubernetes Dashboard
Luke

Score: 33.70974, Course: machine-learning-zoomcamp, Question: How do I copy files from a different folder into docker container’s working directory?
Text: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
In the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows:
COPY ["src/predict.py", "models/xgb_mode

Score: 44.50556

### Q4. Filtering

Now ask a different question: "How do copy a file to a Docker container?".

This time we are only interested in questions from `machine-learning-zoomcamp`.

Return 3 results. What's the 3rd question returned by the search engine?

* How do I debug a docker container?
* How do I copy files from a different folder into docker container’s working directory?
* How do Lambda container images work?
* How can I annotate a graph?

In [10]:
query = "How do copy a file to a Docker container?"

In [11]:
def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [12]:
docs = elastic_search(query)
for doc in docs:
    print(f"Course: {doc['course']}, Section: {doc['section']}")
    print(f"Question: {doc['question']}")
    print(f"Text: {doc['text']}\n")

Course: machine-learning-zoomcamp, Section: 5. Deploying Machine Learning Models
Question: How do I debug a docker container?
Text: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Course: machine-learning-zoomcamp, Section: 5. Deploying Machine Learning Models
Question: How do I copy files from my local machine to docker container?
Text: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani

Course: machine-learn

3rd question returned by the search engine

Question: How do I copy files from a different folder into docker container’s working directory?

### Q5. Building a prompt

Now we're ready to build a prompt to send to an LLM. 

Take the records returned from Elasticsearch in Q4 and use this template to build the context. Separate context entries by two linebreaks (`\n\n`)
```python
context_template = """
Q: {question}
A: {text}
""".strip()
```

Now use the context you just created along with the "How do I execute a command in a running docker container?" question 
to construct a prompt using the template below:

```
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
```

What's the length of the resulting prompt? (use the `len` function)

* 946
* 1446
* 1946
* 2446

In [28]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context_template = """
Q: {question}
A: {text}
""".strip()
    context = ""
    
    for doc in search_results:
        context += context_template.format(question=doc['question'], text=doc['text']) + "\n\n"
    # context = context.strip()
        # context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [29]:
prompt = build_prompt(query, docs)
print(prompt)
print("Length of the prompt:", len(prompt))

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do copy a file to a Docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/contain

Length of the resulting prompt is 1446

### Q6. Tokens

When we use the OpenAI Platform, we're charged by the number of 
tokens we send in our prompt and receive in the response.

The OpenAI python package uses `tiktoken` for tokenization:

```bash
pip install tiktoken
```

Let's calculate the number of tokens in our query: 

```python
encoding = tiktoken.encoding_for_model("gpt-4o")
```

Use the `encode` function. How many tokens does our prompt have?

* 120
* 220
* 320
* 420

Note: to decode back a token into a word, you can use the `decode_single_token_bytes` function:

```python
encoding.decode_single_token_bytes(63842)
```

In [30]:
from openai import OpenAI
import tiktoken
client = OpenAI()

In [ ]:
def num_tokens_from_string(string: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.encoding_for_model("gpt-4o")
    num_tokens = len(encoding.encode(string))
    return num_tokens
# get number of tokens in the prompt
num_tokens = num_tokens_from_string(prompt)
print(f"Number of tokens in the prompt: {num_tokens}")

Number of tokens in the prompt: 320


Number of tokens in the prompt: 320

In [40]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

response = llm(prompt)
print("LLM response:")
print(response)

LLM response:
To copy a file to a Docker container from your local machine, you can use the `docker cp` command. The basic syntax is as follows:

```shell
docker cp /path/to/local/file_or_directory container_id:/path/in/container
```


In [39]:
response

'You can copy a file to a Docker container using the `docker cp` command. The basic syntax for copying a file or directory from your local machine to a running Docker container is:\n\n```\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\n```'